# Apache Drill - Querying The Yelp Dataset

In [1]:
from pydrill.client import PyDrill
from itertools import chain
from collections import Counter
import pandas as pd

## Connect to PyDrill 

In [2]:
drill = PyDrill(host="localhost", port=8047)

## Tutorial - Query Yelp Business JSON Data 

Perform a query to return one row from the `yelp_academic_dataset_business.json` dataset.

In [3]:
drill.query('''SELECT * FROM dfs.`drill/data/yelp_academic_dataset_business.json` limit 1 ''').to_dataframe()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state
0,1314 44 Avenue NE,"{""BikeParking"":""False"",""BusinessAcceptsCreditC...",Apn5Q_b6Nz61Tq4XzPdf9A,"Tours, Breweries, Pizza, Restaurants, Food, Ho...",Calgary,"{""Monday"":""8:30-17:0"",""Tuesday"":""11:0-21:0"",""W...",1,51.0918130155,-114.031674872,Minhas Micro Brewery,,T2E 6L6,24,4.0,AB


### Query - Total Businesses in Dataset

Review how many rows are in the entire dataset. You should have ~189,000 rows of data.

In [4]:
drill.query('''SELECT count(*) AS total_businesses
    FROM dfs.`drill/data/yelp_academic_dataset_business.json`''').\
    to_dataframe()

,total_businesses
0,188593


###  Query - Total Open Businesses

We know how many businesses there are from the query above, but now we want to see how many are still in business today. Perform the SQL query below with PyDrill. The aggregation will return a count of how many businesses are open (1) and closed (0).

In [5]:
drill.query('''SELECT is_open, count(*) AS total_businesses
    FROM dfs.`drill/data/yelp_academic_dataset_business.json` group by is_open ''').\
    to_dataframe()

,is_open,total_businesses
0,0,31987
1,1,156606


### Query - Count Business Categories 

Let's look at the total business categories in the dataset. You may have noticed from the first query `categories` is filled with more than one value. The following query will count all occurances of each business category in the data set.

In [6]:
df_categories = drill.query('''SELECT categories
    FROM dfs.`drill/data/yelp_academic_dataset_business.json` ''').\
    to_dataframe()

Print the output of the query. Notice the entire output is a single string.

In [7]:
df_categories["categories"].loc[0]

'Tours, Breweries, Pizza, Restaurants, Food, Hotels & Travel'

To separate each category into a unique string, we'll split each string by the `,` delimiter.

In [8]:
df_categories["categories"] = df_categories["categories"].str.split(",")

We'll now convert the `categories` Series to a list. This is the expected format we'll need, and also is faster to process than a Pandas Series.

After creating the list, we'll view the output.

In [9]:
lst_categories = df_categories["categories"].tolist()
lst_categories[0:5]

[['Tours',
  ' Breweries',
  ' Pizza',
  ' Restaurants',
  ' Food',
  ' Hotels & Travel'],
 ['Chicken Wings',
  ' Burgers',
  ' Caterers',
  ' Street Vendors',
  ' Barbeque',
  ' Food Trucks',
  ' Food',
  ' Restaurants',
  ' Event Planning & Services'],
 ['Breakfast & Brunch', ' Restaurants', ' French', ' Sandwiches', ' Cafes'],
 ['Insurance', ' Financial Services'],
 ['Home & Garden',
  ' Nurseries & Gardening',
  ' Shopping',
  ' Local Services',
  ' Automotive',
  ' Electronics Repair']]

Notice the output is a list of lists. To get a total count, this must be flattened to a single list.

In [10]:
# Remove blank lists where no business categories were given.
lst_categories = list(filter(None, lst_categories))

# Flatten the list of lists into a single list.
lst_categories = list(chain(*lst_categories))

lst_categories[0:10]

['Tours',
 ' Breweries',
 ' Pizza',
 ' Restaurants',
 ' Food',
 ' Hotels & Travel',
 'Chicken Wings',
 ' Burgers',
 ' Caterers',
 ' Street Vendors']

The next step is to remove the leading whitespace from the strings.

In [11]:
lst_categories = [element.lstrip() for element in lst_categories]
lst_categories[0:10]

['Tours',
 'Breweries',
 'Pizza',
 'Restaurants',
 'Food',
 'Hotels & Travel',
 'Chicken Wings',
 'Burgers',
 'Caterers',
 'Street Vendors']

Finally, we can get a count of the most common restaurant categores using the `Counter` method. This only returns the top 10 most common categories, but you can use any number you'd like.

In [12]:
Counter(lst_categories).most_common(10)

[('Restaurants', 57173),
 ('Shopping', 30231),
 ('Food', 27118),
 ('Beauty & Spas', 18967),
 ('Home Services', 18634),
 ('Health & Medical', 16157),
 ('Local Services', 12906),
 ('Automotive', 12656),
 ('Nightlife', 12438),
 ('Bars', 10853)]

### Query - Ratings Analysis  

Review the average rating, along with the total ratings per business.

In [13]:
df_ratings = drill.query('''SELECT name, count(*) AS total_ratings, avg(stars) AS avg_rating
    FROM dfs.`drill/data/yelp_academic_dataset_business.json` group by name order by avg_rating desc''').\
    to_dataframe()

df_ratings.head()

,avg_rating,name,total_ratings
0,5.0,Alternative Portable Buildings,1
1,5.0,Core Revolution,1
2,5.0,Freedom Firearms,1
3,5.0,Hillary Salon,1
4,5.0,Fast Service Movers,1


Review businesses that have the most ratings.

In [14]:
max_ratings = df_ratings["total_ratings"].max()

df_ratings[df_ratings["total_ratings"] == max_ratings]

,avg_rating,name,total_ratings
104023,3.2653061224489797,Dollar Tree,98
131160,2.311224489795918,Pizza Pizza,98


Now, let's look at businesses with the highest ratings. Notice many businesses have a rating of `5.0`, but only have one rating.

In [15]:
highest_rating = df_ratings["avg_rating"].max()

df_ratings[df_ratings["avg_rating"] == highest_rating].head(10)

,avg_rating,name,total_ratings
0,5.0,Alternative Portable Buildings,1
1,5.0,Core Revolution,1
2,5.0,Freedom Firearms,1
3,5.0,Hillary Salon,1
4,5.0,Fast Service Movers,1
5,5.0,Barber Shop at Carefree,1
6,5.0,Hillside Animal Hospital,1
7,5.0,Brief,1
8,5.0,Madeleine Bakery And Bistro,1
9,5.0,Phat Daddy's Kitchen,1
